In [1]:
from astropy.io import fits
from astropy import units as u
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from astrodendro import Dendrogram, pp_catalog, ppv_catalog
from astrodendro.analysis import PPStatistic, PPVStatistic
from astrodendro.scatter import Scatter

import numpy as np
import scipy.ndimage as nd
import fileinput
import os
from os.path import expanduser

In [ ]:
'''
# Expand the tilde to the full path
file_path = expanduser('~/miniconda3/envs/aplpy/lib/python3.9/site-packages/astrodendro/analysis.py')

# Ensure the file exists before attempting to modify it
if not os.path.exists(file_path):
    raise FileNotFoundError(f"The file {file_path} does not exist.")

# Replace all instances of np.int with int
with fileinput.FileInput(file_path, inplace=True, backup='.bak') as file:
    for line in file:
        print(line.replace('np.asscalar', 'np.ndarray.item'), end='')
'''

In [3]:
# Constants
c = 2.99792458e+10  # Speed of light in cm/s
h = 6.626176e-27    # Planck constant in cm^2 g/s
kb = 1.380658e-16   # Boltzmann constant in erg/K
pc2cm = 3.0857e+18  # Parsec in cm
OneJy = 1.0e-23     # Jansky in erg/s/cm^2/Hz
Msun = 2.0e+33      # Solar mass in gm

def planck(nu, Temp):
    wav = c / nu
    aa = 2.0 * h * c ** 2
    bb = h * c / (wav * kb * Temp)
    intensity = aa / ((wav ** 5) * (np.exp(bb) - 1.0))
    intensity = wav * wav * intensity / c
    return intensity

def CalculateMassFromIntensity(Tdust, nu, Fnu, dist, beta, eta):
    dist = dist * 1000. * pc2cm
    kappa = 10 * ((nu / 1.2e+12) ** beta)  # beta = Spectral index
    Fnu = Fnu * OneJy
    Bnu = planck(nu, Tdust)
    M = eta * Fnu * (dist ** 2) / (kappa * Bnu)
    return M / Msun

def process_fits_file(fits_file, distance):
    # Parameters
    Tdust = 20.0
    Gas2Dust = 100.0
    beta = 1.5
    rms = 0.05 # from statistics
    bg = 0 # median or mean value
    min_val = bg + rms    
    min_delt = rms * 5 # 5 sigma
    min_npix = 9 # nyquist
    FoV = 25  # in arcsec # take from ds9 by drawing a region, or from header naxis*cdelt
    with fits.open(fits_file) as hdul:
            data = hdul[0].data
            header = hdul[0].header
    Obsfreq = header['RESTFRQ'] # in Hz

    OutputExtn = os.path.splitext(os.path.basename(fits_file))[0]

    # Open the FITS file
    hdu = fits.open(fits_file)[0]
    w = WCS(fits_file).celestial

    xlen = np.shape(hdu.data)[2]
    ylen = np.shape(hdu.data)[3]

    data = hdu.data[0, 0, :]

    PixSize = w.wcs.cdelt[1] * 3600.0
    FoVPix = int(FoV / (2 * PixSize))
    bmaj = float(hdu.header['BMAJ']) * 3600.0
    bmin = float(hdu.header['BMIN']) * 3600.0

    metadata = {
        'data_unit': u.Jy / u.beam,
        'spatial_scale': PixSize * u.arcsec,
        'beam_major': bmaj * u.arcsec,
        'beam_minor': bmin * u.arcsec,
        'wcs': w
    }

    # Compute the dendrogram
    d = Dendrogram.compute(data, min_value=min_val, min_delta=min_delt, min_npix=min_npix)
    print(f'Number of identified Sources: {len(d.leaves)}')

    # Ensure all directories for the output files exist
    clump_details_dir = os.path.dirname(f'ClumpDetails_{OutputExtn}.cat')
    ds9_dir = os.path.dirname(f'ds9_{OutputExtn}.reg')
    casa_dir = os.path.dirname(f'Casa_{OutputExtn}.reg')

    for directory in set([clump_details_dir, ds9_dir, casa_dir]):
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

    # Open output files
    with open(f'ClumpDetails_{OutputExtn}.cat', 'w') as fo, \
         open(f'ds9_{OutputExtn}.reg', 'w') as fo1, \
         open(f'Casa_{OutputExtn}.reg', 'w') as fo2:
        
        fo.write(f'# Distance: {distance:.1f} kpc; Min Good Value: {min_val:.4f}; Min Delta: {min_delt:.4f}; Min No. of Pix: {min_npix}\n')
        fo.write(f'# Tdust = {Tdust:.1f} K; Gas-to-dust = {Gas2Dust:.1f}\n')
        fo.write('#    RA          Dec      FWHMx(")  FWHMy(")    PA       R(") Area(sq-")   Flux           MSun\n')
        fo.write('#  (deg)        (deg)                                                      (Jy)     beta=1.5  beta=2.0\n')
        
        fo1.write('global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\nfk5\n')
        fo2.write('#CRTFv0 CASA Region Text Format version 0\n')

        index = 0
        for leaf in d.leaves:
            s = PPStatistic(leaf, metadata=metadata)
            x = s.x_cen
            y = s.y_cen
            FWHMx = 2. * s.major_sigma
            FWHMy = 2. * s.minor_sigma
        
            R = 2. * s.radius
            Area = s.area_exact
            Flux = s.flux  # Ensure this is in Jy from your metadata definition
            PA = s.position_angle

            ra, dec = x, y
            xcen, ycen = w.all_world2pix(ra, dec, 1)
            Rpix = R / PixSize
            AreaPix = Area / (PixSize * PixSize)

            if ((xcen >= (xlen/2. - FoVPix)) and (xcen <= (xlen/2. + FoVPix)) and
                (ycen >= (ylen/2. - FoVPix)) and (ycen <= (ylen/2. + FoVPix))):
                
                # Ensure Flux is in Jy
                Flux_Jy = Flux.to(u.Jy).value
                
                M20_1 = CalculateMassFromIntensity(Tdust, Obsfreq, Flux_Jy, distance, 1.5, Gas2Dust)
                M20_2 = CalculateMassFromIntensity(Tdust, Obsfreq, Flux_Jy, distance, 2.0, Gas2Dust)
        
                fo.write('%11.6f %11.6f  %8.3f %8.3f %8.2f  %8.3f %8.3f  %8.4e  %6.2f    %6.2f\n'% \
                        (ra, dec, FWHMx, FWHMy, PA.value,  R, Area, Flux_Jy, M20_1, M20_2))
                fo1.write('ellipse( %11.6f, %11.6f, %8.3f", %8.3f", %8.3f)\n'%(ra, dec, FWHMx, FWHMy, PA.value))
                fo2.write('ellipse [[%12.7fdeg, %12.7fdeg], [%8.4farcsec, %8.4farcsec], %8.3fdeg] coord=ICRS, corr=[I], linewidth=1, linestyle=-, symsize=1, symthick=1, color=magenta, font=Ubuntu, fontsize=11, fontstyle=normal, usetex=false\n'%\
                         (ra, dec, FWHMx, FWHMy, PA.value))

                index = index + 1

    print(f'Finally selected Sources: {index}')

In [4]:
fits_file = '/media/sanjana/One Touch/Toolkit/OutflowProject/ALMAData/I14498_Final_mfs.image.pbcor.fits'
distance = 3.2
process_fits_file(fits_file, distance)

Number of identified Sources: 1
Finally selected Sources: 1


Set OBSGEO-B to   -23.022886 from OBSGEO-[XYZ].
Set OBSGEO-H to     5053.796 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [5]:
fits_file = '/media/sanjana/One Touch/Toolkit/OutflowProject/ALMAData/I16060_Final_mfs.image.pbcor.fits'
distance = 5.2
process_fits_file(fits_file, distance)

Number of identified Sources: 2
Finally selected Sources: 2


Set OBSGEO-B to   -23.022886 from OBSGEO-[XYZ].
Set OBSGEO-H to     5053.796 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
